# Import Required Libraries

In [1]:
import re
import unicodedata
from collections import defaultdict, Counter
import networkx as nx
from itertools import combinations
import difflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import jdatetime

### Load Dataset

In [2]:
df = pd.read_csv('reviews.csv')

### Basic Cleaning

In [3]:
"""
During EDA, we determined that nearly all missing values were due to: 
survey images, professor suggestion polls, or various single messages 
(like pinned posts, guides, or initial channel messages).
We may revisit EDA-related code later to extract any potentially useful information from these if possible.
For now, we removing them as we did during the EDA phase.
"""

# Just 2 Special Cases
df.loc[df['message_id'] == 816, ['department', 'comment_text', 'parse_error']] = [
    'معارف', 
    'خییلللللی استاد محشرین از دسشون ندین ۴ نمره اضافی دارن برای نهاد که جواباش پیدا میشه😁 امتحاناشونم میانترم کپی ترم پیش و پایانترم از یه فایل ۲۰ صفحه ای بود',
    None
]

df.loc[df['message_id'] == 252, 'comment_text'] = np.nan

df = df.dropna(subset=['term'])

In [4]:
print("Total reviews:", len(df))
print("Unique professors:", df["professor_name_raw"].nunique())
print("Departments:", df["department"].nunique())

Total reviews: 4276
Unique professors: 1296
Departments: 18


In [5]:
df

,message_id,date,term,professor_id,professor_name_raw,department,course_name,coherence,knowledge,teaching,management,responsiveness,behavior,grading_status_raw,attendance_status_raw,comment_text,parse_error
3,14,2021-09-05 00:34:39,مهر 99,NaN,سید محمد شهرتاش,برق,بررسی ۱-بررسی۲-حفاظت,8.0,9.0,7.0,9.0,8.0,10.0,منصفانه و هرچی خودت بگیری,حضور و غیاب نمی کند,چیزی اضافه ایی نیست,NaN
4,15,2021-09-05 00:35:51,بهمن 98,NaN,دکتر هاجر فلاحتی,مهندسی_کامپیوتر,مدار منطقی- طراحی سیستم های کامپیوتری,1.0,2.0,1.0,3.0,2.0,10.0,نمره خوبی نمیشه ازشون گرفت,حضور مهم نیست اما تاثیر مثبت دارد,خوب درس نمیده اصلا، نمره ها رو خوب نمیده و اصل...,NaN
5,16,2021-09-05 00:36:56,بهمن 99,NaN,علم الهدی,مهندسی_مواد,بلور شناسی,9.0,8.0,10.0,9.0,10.0,10.0,منصفانه و هرچی خودت بگیری,حضور و غیاب نمی کند,در کل اگر که دنبال یک استاد با ادب با دانشجو م...,NaN
6,17,2021-09-05 00:37:00,مهر 98,NaN,حجت قاسمی,مکانیک,سوخت و احتراق,10.0,10.0,9.0,9.0,9.0,9.0,منصفانه و هرچی خودت بگیری,حضور و غیاب نمی کند,سخت گیر و پربازده,NaN
7,18,2021-09-05 00:41:22,بهمن 99,NaN,فریبرز قاسمی,مهندسی_کامپیوتر,برنامه نویسی کامپیوتر,8.0,10.0,8.0,10.0,7.0,9.0,منصفانه و هرچی خودت بگیری,حضور و غیاب نمی کند,چیزی اضافه ایی نیست,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4591,4723,2025-09-27 14:55:48,بهمن 1403,NaN,سید حمیدرضا نوشکاران,ریاضی,معادلات دیفرانسیل,8.0,8.0,8.0,9.0,8.0,10.0,منصفانه و هرچی خودت بگیری,حضور مهم نیست اما تاثیر مثبت دارد,ترم اولی بود که تدریس می‌کردن ولی تدریس بدی ند...,NaN
4592,4724,2025-10-10 14:51:17,بهمن ۱۴۰۳,NaN,عین الله خنجری,مهندسی_کامپیوتر,پایگاه داده پیشرفته,8.0,6.0,4.0,6.0,1.0,3.0,نمره خوبی نمیشه ازشون گرفت,حضور و غیاب نمی کند,تو نمره دادن پروژه‌ها بسیار رندوم و بی‌منطق عم...,NaN
4593,4725,2025-10-10 14:51:28,بهمن ۱۴۰۳,NaN,رضا انتظاری ملکی,مهندسی_کامپیوتر,مباحث ویژه نرم افزار ۱,10.0,10.0,10.0,10.0,10.0,10.0,منصفانه و هرچی خودت بگیری,حضور مهم است و تاثیر مستقیم دارد,استاد بسیار خوش اخلاق و محترمی هستند. تعداد تم...,NaN
4594,4726,2025-10-10 14:51:31,بهمن 1401,NaN,دکتر بخشایش,مهندسی_مواد,مواد پیشرفته,10.0,10.0,10.0,10.0,10.0,10.0,منصفانه و هرچی خودت بگیری,حضور مهم است و تاثیر مستقیم دارد,استاد مطلعی هستند و اطلاعات به روزی دارند.,NaN


In [6]:
df['comment_text'].isna().sum()

np.int64(16)

In [7]:
df.loc[df['department'] == 'شیمی', 'department'] = 'مهندسی_شیمی'

### Term Normalization

In [8]:
temp_grouped = {
 ('قبل از سال 94',): 'before_94',

 ('بهمن 94',): 'second_1394',

 ('تابستان 95',): 'summer_1395',
 
 ('مهر 96',): 'first_1396',
 ('تابستان 96',): 'summer_1396',
 
 ('مهر 97',): 'first_1397',
 ('بهمن 97',): 'second_1397',
 
 ('مهر 98',): 'first_1398',
 ('بهمن 98',): 'second_1398',
 
 ('مهر 99',): 'first_1399',
 ('بهمن 99', 'بهمن ۹۹'): 'second_1399',
 ('تابستان 99',): 'summer_1399',
 
 ('مهر 00', 'مهر 1400', 'مهر ۰۰', 'مهر ..', 'مهر ١٤٠٠', 'مهر ۱۴۰۰', 'مهر۱۴۰۰', 'پاییز ۱۴۰۰'): 'first_1400',
 ('4002', 'بهمن 1400', 'بهمن ۱۴۰۰'): 'second_1400',
 ('تابستان 1400', 'تابستان۱۴۰۰'): 'summer_1400',
 ('بهمن 99 ،مهر 1400', 'مهر و بهمن 1400', 'مهر و بهمن ۱۴۰۰', 'هم ترم مهر ۱۴۰۰ و ترم بهمن ۱۴۰۰'): 'both_1400',
 
 ('14011', 'ترم های 4001 - 4002 - 4011', 'مهر 1401', 'مهر ۴۰۱', 'پاییز ۱۴۰۱'): 'first_1401',
 ('بهار 1402', 'بهار ۱۴۰۲', 'بهار ۴۰۲', 'بهمن 1401', 'بهمن ۴۰۱', 'فرودین ۱۴۰۲'): 'second_1401',
 ('تابستان 1401',): 'summer_1401',
 ('مهر 1401 و بهمن 1402',): 'both_1401',

 ('1402 پاییز', 'مهر 1402', 'مهر 402', 'مهر ۱۴۰۲', 'مهر ۴۰۲', 'مهر۱۴۰۲', 'مهر۴۰۲', 'مهر 1402 - بهمن 1402', 'مهر ۱۴۰۲-بهمن ۱۴۰۲', 'بهمن۱۴۰۱_مهر۱۴۰۲', 'پاییز  ۱۴۰۲', 'پاییز 1402', 'پاییز 402', 'پاییز ۱۴۰۲', '۱۴۰۲ مهر', '۱۴۰۲-۱'): 'first_1402',
 ('4022', 'اسفند 1402', 'اسفند ۱۴۰۲', 'بهار 1403', 'بهار ۱۴۰۳', 'بهار ۴۰۳', 'بهار 403', 'بهتر ۴۰۳', 'بهمن ۱۰۴۲', 'بهمن ۴۰۲ ۴۰۳', 'بهمن۴۰۲-۳', 'بهمن 1402', '1402 بهمن', '1402بهمن', 'بهمن  1402', 'بهمن 1402-1403', 'بهمن 402', 'بهمن ۱۴۰۲', 'بهمن ۴۰۲', 'بهمن۱۴۰۲', 'ترم دو ۱۴۰۲_۱۴۰۳', 'زمستان ۱۴۰۲', 'زمستان ۴۰۲', 'زمستان 402', 'خرداد ۴۰۳', 'زمستان ۱۴۰۲ و بهار ۱۴۰۳', 'زمستان ۴۰۳', '۱۴۰۲٫۱۴۰۳', '۱۴۰۲_۱۴۰۳', '۴۰۲.۲'): 'second_1402',
 ('تابستان 1402',): 'summer_1402',
 ('1402', '۱۴۰۲', 'اول و دوم ۱۴۰۲', 'مهر 1402 و بهمن 1402', 'مهر و بهمن ۴۰۲', '۱۴۰۲ ۱۴۰۳'): 'both_1402',
 
 ('1403 مهر', '403', '4031', 'مهر 1403', 'مهر 403', 'مهر ۱۴٠۳', 'مهر ۱۴۰۳', 'مهر ١۴٠٣', 'مهر ۴۰۳', 'مهر1403', 'مهر403', 'مهر۱۴۰۳', 'مهر۴۰۳', 'مهر۴۰۳- بهمن۴۰۳', 'نیمه اول ۱۴۰۳', 'پائیز 1403', 'پاییز 1403', 'پاییز 403', 'پاییز ۱۴۰۳', 'پاییز ۴۰۳', 'پاییز۴۰۳'): 'first_1403',
 ('1403-2', '4032', '403_2', 'بهار ۱۴۰۴', 'بهار ۴۰۴','بهمن 1403', 'بهمن 403', 'بهمن ١٤٠٣', 'بهمن ۱۴٠۳', 'بهمن ۱۴۰۳', 'بهمن ۴۰۳', 'بهمن۱۴۰۳', 'بهمن ۴۰۲-۳', 'بهمن۴۰۳', 'ترم زمستان 1403', '۴۰۳-۲', '۴۰۳۲'): 'second_1403',
 ('تابستان 1403', 'تابستان 403', 'تابستان ۱۴۰۳', 'تابستان ۴۰۳'): 'summer_1403',
 ('۱۴۰۳', '۴۰۳', 'آبان و بهمن ۱۴۰۳', 'بهمن ۱۴۰۳ و مهر ۱۴۰۳', 'مهر و بهمن 1403', 'مهر و بهمن ۴۰۳', 'مهر ۱۴۰۳ و بهمن ۴۰۳-۴۰۴', 'مهر۱۴۰۳و بهمن ۱۴۰۳'): 'both_1403',

 ('مهر 1404', '۴۰۴'): 'first_1404',
 ('بهمن 1404', 'بهمن ۱۴۰۴', 'بهمن ۴۰۴'): 'second_1404',
 ('تابستان 1404', 'تابستان ۴٠۴'): 'summer_1404'
}

flat_mapping = {
    key: value
    for keys, value in temp_grouped.items()
    for key in keys
}

In [9]:
df['term'] = (
    df['term']
      .astype(str)
      .str.strip()
      .str.replace(r'([آ-یA-Za-z]+)([0-9]+)', r'\1 \2', regex=True)
      .map(flat_mapping))

### Professor Names Normalization

In [10]:
# Checking For Common Prefixes
prefixes = ['سید', 'دکتر', 'مهندس', 'آقای', 'آقا', 'خانم', 'خانوم', 'سرکار', 'سرکار خانم', 'سرکار خانوم', 'استاد', 'پروفسور', 'پرفسور', 'جناب', 'جناب آقای']

pc = {p: 0 for p in prefixes}
for n in df['professor_name_raw']:
    s = str(n).strip()
    for p in prefixes:
        if s.startswith(p):
            pc[p] += 1
            break

for p, c in sorted(pc.items(), key=lambda x: -x[1]):
    if c: print(f"{p}: {c}")

دکتر: 153
سید: 143
استاد: 88
خانم: 2
پروفسور: 2
مهندس: 1
آقای: 1
آقا: 1
پرفسور: 1


In [11]:
PUNCT_RE = re.compile(r'[^\w\sآ-ی]')
VA_RE = re.compile(r'^\s*.+\s+و\s+.+\s*$')

mask = (
    df['professor_name_raw'].fillna('').str.contains(PUNCT_RE.pattern, na=False, regex=True) |
    df['professor_name_raw'].fillna('').str.contains(VA_RE.pattern, na=False, regex=True)
)
combined = df[mask]
combined['professor_name_raw']

324                                   🧑‍
1013              استاد سیدین و ابوطالبی
1946                    اکبری - محمودوند
1981           معصومه احمدی-زهرا عابدینی
2025                خانم احمدی و عابدینی
2066              دکتر زمانی-مهندس احمدی
2118                  نوروزبیگی-سعیدزاده
2445       استاد عابدینی، داوودی و اکبری
2605    استاد چوپانکاره و استاد میرشفیعی
2635          مجید نوروزی - امید محمدیان
2708        مریم صالحی و حمید رضا کوهدار
2890                بوترابی، عشیری، رضوی
3049           علیرضا معینی، صدیقه شریفی
3267       مهرانه تیراندری -زهرا عابدینی
3284                                  🧑‍
3401          دکتر سیدین و دکتر ابوطالبی
3632             گرشاسبی و زهرا بیرانوند
4590                     ساکیانی-اقاجانی
Name: professor_name_raw, dtype: object

In [12]:
inp = 'professor_name_raw'
out = 'prof_merged'

# INITIAL NORMALIZE
CHAR_MAP = {'ي':'ی','ى':'ی','ے':'ی','ك':'ک'}
ZW_RE = re.compile(r'[\u200c\u200d\uFEFF]')

def basic_normalize(s):
    if not isinstance(s, str):
        return ''
    s = unicodedata.normalize('NFKC', s.strip())
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    for a, b in CHAR_MAP.items():
        s = s.replace(a, b)
    s = ZW_RE.sub('', s)
    return re.sub(r'\s+', ' ', s).strip()

# Prefixes Removal
PREFIXES = ['سیده', 'سید', 'مهندس', 'استاد', 'خانم', 'پروفسور', 'پرفسور', 'آقای', 'آقا', 'اقای دکتز']
pat_start = re.compile(r'^\s*(?:' + '|'.join(map(re.escape, PREFIXES)) + r')(?=\s|[آ-ی]|$)\s*')

# Special Case
SPECIAL_CASES = ['ریاضی دو', 'زبان عمومی']
pat_special = re.compile(r'\b(?:' + '|'.join(map(re.escape, SPECIAL_CASES)) + r')\b')

# Special Token Fix
FIXES = [
    (re.compile(r'\bاقا(?=[آ-ی])'), 'آقا'),
    (re.compile(r'\bاشتیانی\b'), 'آشتیانی'),
    (re.compile(r'\bازاد\b'), 'آزاد'),
    (re.compile(r'\bاسمعیلی\b'), 'اسماعیلی'),
    (re.compile(r'\bاللهی\b'), 'الهی'),
    (re.compile(r'الله'), 'اله'),
    (re.compile(r'الله'), 'اله'),
]


def clean_name(s):
    s = basic_normalize(s)

    has_doc = bool(re.search(r'\bدکتر\b', s))
    s = re.sub(r'\bدکتر\b', ' ', s)

    s = pat_special.sub(' ', s)

    while True:
        new = pat_start.sub('', s)
        if new == s:
            break
        s = new

    for p, r in FIXES:
        s = p.sub(r, s)

    s = re.sub(r'\s+', ' ', s).strip()
    return s, has_doc

def format_rep(base, has_doc):
    return ('دکتر ' + base) if (has_doc and base) else base

In [13]:
DEPT_TH = {'فیزیک': 0.75, 'مکانیک': 0.75, 'مهندسی_مواد': 0.80, 'مهندسی_کامپیوتر': 0.71,
'برق': 0.76, 'مهندسی_عمران': 0.71, 'معارف': 0.79, 'مهندسی_شیمی': 0.80, 'ریاضی': 0.72, 'مهندسی_راه_آهن': 0.79,}

def name_similarity(a, b, dept=None):
    a = re.sub(r'\s+', ' ', a.strip())
    b = re.sub(r'\s+', ' ', b.strip())
    if not a or not b:
        return False

    a_tokens = a.split()
    b_tokens = b.split()
    num_a = len(a_tokens)
    num_b = len(b_tokens)

    threshold = DEPT_TH.get(dept, 0.75)

    if num_a == 1 and num_b == 1:
        threshold = 0.85

    is_single_vs_last = False
    if min(num_a, num_b) == 1 and max(num_a, num_b) > 1:
        short_token = a_tokens[0] if num_a == 1 else b_tokens[0]
        long_tokens = b_tokens if num_a == 1 else a_tokens
        if short_token == long_tokens[-1]:
            is_single_vs_last = True

    a_no_space = ''.join(a_tokens)
    b_no_space = ''.join(b_tokens)
    similarity = difflib.SequenceMatcher(None, a_no_space, b_no_space).ratio()

    if is_single_vs_last:
        return True

    return similarity >= threshold

In [14]:
def merge_by_dept(df, inp, out, c=1):
    df[out] = ''

    for department, sub in df.groupby('department'):
        key_to_rows = defaultdict(list)

        for idx, orig in sub[inp].fillna('').items():
            if c == 1:
                base = orig.strip()
                has_doc = False
            else:
                base, has_doc = clean_name(orig)

            if base:
                key_to_rows[base].append((idx, has_doc))

        keys = list(key_to_rows)
        if not keys:
            continue

        G = nx.Graph()
        G.add_nodes_from(keys)

        for a, b in combinations(keys, 2):
            if name_similarity(a, b, department):
                G.add_edge(a, b)

        for cluster in nx.connected_components(G):
            rep_base = max(
                cluster,
                key=lambda k: (len(key_to_rows[k]), len(k.split()))
            )

            doc_present = any(
                has_doc for k in cluster for _, has_doc in key_to_rows[k]
            )

            rep = format_rep(rep_base, doc_present)

            for k in cluster:
                for idx, _ in key_to_rows[k]:
                    df.at[idx, out] = rep

    return df


In [15]:
manual_cases = {
    # فیزیک
    'راه چمنی': 'ذبیح اله راه چمنی', 'راهچمنی': 'ذبیح اله راه چمنی',
    'زهرا عابدینی امین آباد': 'زهرا عابدینی', 
    'زهرا کی احمدی': 'زهرا کی احمدی',

    # مکانیک
    'آیت الهی': 'مجیدرضا آیت الهی',

    # مهندسی مواد
    'رضوی عشیری': 'روح الهی عشیری',

    # مهندسی کامپیوتر
    'زهرا رشیدی': 'زهرا رشیدی', 'رضا انتظار ملکی': 'رضا انتظاری ملکی',

    # برق
    'داوود عرب': 'داوود عرب خابوری', 'عرب': 'داوود عرب خابوری',
    'حسین عرب': 'حسین عرب',
    'رحیمی پور': 'رحیمی پور',
    'راحمی': 'راحمی نوش آبادی',
    'حاجی احمدی': 'محمد جواد حاجی احمدی',
    'توتونچیان': 'فرید توتونچیان',

    # مهندسی عمران
    'علیزاده': 'حسین علیزاده', 'حسین علیزاده': 'حسین علیزاده',
    'علی کاوه': 'علی کاوه',

    # معارف
    'رهبری': 'سعید رهبری شمیرانی', 'سعید رهبری': 'سعید رهبری شمیرانی',
    'هوشمند': 'دکتر نرگس هوشمند همدانی', 'نرگس هوشمند': 'دکتر نرگس هوشمند همدانی', 'دکتر نرگس هوشمند': 'دکتر نرگس هوشمند همدانی',
    'حسن محسنی': 'حسن محسنی',
    'شاه علی': 'احمدرضا شاه علی',
    'تاج الدین': 'علی محمد تاج الدین',
    'هادی نوراله': 'نوراله هادی',
    'محمد فارسانی امینی': 'محمد امینی فارسانی',

    # مهندسی شیمی
    'احمد رهبر': 'احمد رهبر کلیشمی', 'رهبر': 'احمد رهبر کلیشمی',

    # ریاضی
    'محمدهادی علاییان': 'محمد هادی علائیان', 'محمدهادی علائیان': 'محمد هادی علائیان',  'محمد هادی علائیان': 'محمد هادی علائیان',
    'جانی': 'اسداله آقاجانی',
    'تومانیان مگردیچ': 'مگردیچ تومانیان',

    # مهندسی راه آهن
    'حمید رضا احدی': 'حمیدرضا احدی', 'حمیدرضا احدی': 'حمیدرضا احدی',
    'حمیدرضا حیدری': 'حمیدرضا حیدری', 'حمیدرضا حیدری نوقابی': 'حمیدرضا حیدری',
    'امین اوحدی اصفهانی': 'امین اوحدی',
}

In [16]:
def print_clusters_by_department(df, inp, out, dept=None, c=1):

    if c == 0:
        if dept is None:
            print("Please provide a department for c=0 mode.")
            return
        sub = df[df['department'] == dept]
        if sub.empty:
            print(f"No data for department: {dept}")
            return
        print(f"Department: {dept}")
    else:
        sub = df
        print("All data (clustered ignoring department)")

    cluster_to_members = defaultdict(list)

    for _, row in sub.iterrows():
        if c == 1:
            base = str(row[inp]).strip()
        else:
            base, _ = clean_name(row[inp])

        if base:
            cluster_to_members[row[out]].append(base)

    for rep, members in sorted(cluster_to_members.items()):
        unique_members = sorted(set(members))
        print(f"{unique_members} : {rep}")


In [17]:
dfn = df[~mask].copy()
dfn = merge_by_dept(dfn, inp, out)
inv = {v: k for k, v in manual_cases.items()}
dfn[out] = dfn['professor_name_raw'].map(inv).fillna(dfn[out])

In [18]:
dfn['department'].unique()

array(['برق', 'مهندسی_کامپیوتر', 'مهندسی_مواد', 'مکانیک', 'فیزیک',
       'ریاضی', 'مهندسی_صنایع', 'مهندسی_شیمی', 'معارف', 'مهندسی_عمران',
       'تربیت_بدنی', 'مهندسی_راه_آهن', 'معماری_و_شهرسازی', 'طراحی_صنعتی',
       'مهندسی_خودرو', 'مهندسی_پیشرفت', 'فناوری_های_نوین'], dtype=object)

In [19]:
print_clusters_by_department(dfn, inp, out, 'ریاضی')

All data (clustered ignoring department)
['آتنا بروغنی'] : آتنا بروغنی
['آذری'] : آذری
['آرش سراحیان'] : آرش سراحیان
['آرش کیومرثی'] : آرش کیومرثی
['آرمیتا زاهدی'] : آرمیتا زاهدی
['آرنگ پژوهشگر'] : آرنگ پژوهشگر
['آزاده تجردی'] : آزاده تجردی
['آسیه سادات کاظمی'] : آسیه سادات کاظمی
['آسیه نمازی', 'نمازی'] : آسیه نمازی
['آقای حسین شیرانی', 'حسین شیرانی'] : آقای حسین شیرانی
['آیت اللهی'] : آیت اللهی
['مجیدرضا آیت الهی'] : آیت الهی
['آبت قره قانی', 'آیا قره قانی', 'آیت قره قانی', 'ایت قره قانی', 'قره قانی'] : آیت قره قانی
['ابراهیم اردشیر لاریجانی', 'اردشیر لاریجانی'] : ابراهیم اردشیر لاریجانی
['ابراهیم تیموری'] : ابراهیم تیموری
['ابن الرسول'] : ابن الرسول
['ابلفضل اسکندری', 'ابوالفضل اسکندری', 'ابولفضل اسکندری'] : ابوالفضل اسکندری
['ابوالفضل حسینی زاده', 'سید ابوالفضل حسینی زاده', 'سید ابولفضل حسینی زاده'] : ابوالفضل حسینی زاده
['ابوالفضل دیانت'] : ابوالفضل دیانت
['ابوذر غفاری', 'ابوذرغفاری'] : ابوذر غفاری
['احد قائمی'] : احد قائمی
['احسان اکبری'] : احسان اکبری
['احسان دارستانی', 'احسان دا

In [20]:
# Final Normalization
def align_merge(df, merged='prof_merged', out1='dept', out2='professor'):
    df = df.copy()
    parsed = df[merged].map(clean_name)
    df['_base'] = parsed.map(lambda x: x[0])
    df['_has_doc'] = parsed.map(lambda x: x[1])

    df['_key'] = df['_base'].map(lambda x: ''.join(x.split()) if x else '')

    key_to_dept = (
        df.groupby('_key')['department']
        .apply(lambda x: Counter(x).most_common(1)[0][0])
        .to_dict()
    )

    key_has_doc = df.groupby('_key')['_has_doc'].any().to_dict()

    key_to_base = (
        df.groupby('_key')['_base']
        .apply(lambda x: Counter(x).most_common(1)[0][0])
        .to_dict()
    )

    df[out1] = df['_key'].map(key_to_dept)
    df[out2] = df['_key'].map(lambda k: format_rep(key_to_base[k], key_has_doc[k]))

    return df.drop(columns=['_base', '_has_doc', '_key'])

In [21]:
dfnn = align_merge(dfn)

In [22]:
changed = dfnn[dfnn['department'] != dfnn['dept']].drop_duplicates(subset=['prof_merged'])
changed[['professor_name_raw', 'prof_merged', 'professor', 'department', 'dept']]

,professor_name_raw,prof_merged,professor,department,dept
127,سعیدرضا مساح,سعیدرضا مساح,سعیدرضا مساح,مهندسی_راه_آهن,مهندسی_عمران
189,محمد شهبازی,محمد شهبازی,محمد شهبازی,مکانیک,معارف
192,وحید خطیبی,وحید خطیبی,وحید خطیبی,مهندسی_کامپیوتر,مهندسی_صنایع
256,هادی نوراله,هادی نوراله,هادی نوراله,برق,معارف
258,جواد غفاری,جواد غفاری,جواد غفاری,مهندسی_راه_آهن,معارف
283,رضا انتظار ملکی,رضا انتظاری ملکی,رضا انتظاری ملکی,برق,مهندسی_کامپیوتر
312,صادق رستم نیا,صادق رستم نیا,صادق رستم نیا,مهندسی_مواد,مهندسی_شیمی
333,محمد امینی فارسانی,محمد فارسانی امینی,محمد فارسانی امینی,مکانیک,معارف
368,محمود میرطباطبایی,محمود میرطباطبایی,محمود میرطباطبایی,مهندسی_عمران,معارف
371,محمدعلی شفیعا,محمدعلی شفیعا,دکتر محمد علی شفیعا,مهندسی_صنایع,مهندسی_پیشرفت


In [23]:
dfnn['professor_id'] = pd.factorize(dfnn['professor'])[0] + 1

### Grading & Attendence Categorization 

In [24]:
def map_grading(s):
    s = (s or "").strip().lower()
    unknown_kw = ['رندوم', 'رندم', 'شانسی', 'تاس', 'بدون منطق', 'نامشخص', 'نمیدونم', 'یادم نیست', 'کاملا شانسی', 'اصلا قابل']
    fair_kw    = ['منصف', 'مطلوب', 'عادلانه', 'انصاف']
    easy_kw    = ['آسان', 'راحت', 'دست باز', 'ارفاق', 'هرچی خودت', 'راحت میشه', 'عشقی', 'شیفت', 'ارفاقی']
    strict_kw  = ['سخت', 'سختگیر', 'سخت‌گیر', 'سختگیرانه', 'نمره خوبی نمیشه', 'کمتر از', 'ازحقت', 'افتضاح', 'بد']
    
    if any(k in s for k in unknown_kw):
        return "نامشخص"
    if any(k in s for k in fair_kw):
        return "منصفانه"
    if any(k in s for k in easy_kw):
        return "آسان"
    if any(k in s for k in strict_kw):
        return "سخت گیر"
    return "نامشخص"

dfnn['grading_status'] = dfnn['grading_status_raw'].fillna('').astype(str).apply(map_grading)


attendance_map = {
    'حضور و غیاب نمی کند': 'آزاد',
    'حضور مهم نیست اما تاثیر مثبت دارد': 'متوسط',
    'حضور مهم است و تاثیر مستقیم دارد': 'سخت گیر',
    'یادم نمیاد': 'نامشخص'
}
dfnn['attendance_status'] = dfnn['attendance_status_raw'].map(attendance_map).fillna('نامشخص')

### Final Save Operation 

In [25]:
final_cols = ['term', 'professor_id', 'professor', 'dept', 'coherence', 'knowledge', 'teaching',
    'management', 'responsiveness', 'behavior', 'comment_text', 'attendance_status', 'grading_status']
final = dfnn[final_cols].copy()

final.to_csv('cleaned.csv', index=False, encoding='utf-8-sig')